# 🎓 Proiektu Finala - Deep Computer Vision

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Proiektuaren Helburua

Notebook honetan **Computer Vision proiektu integral** bat garatuko dugu:
- Dataset bat aukeratu
- CNN eredua entrenatu (Transfer Learning-ekin)
- Objektu detekzioa aplikatu
- Emaitzak aztertu eta aurkeztu

---

## 📚 Proiektuaren Faseak

1. ✅ Dataset-a aukeratu eta kargatu
2. ✅ Datu esplorazioa eta pre-prozesatzea
3. ✅ CNN eredua entrenatu (Transfer Learning)
4. ✅ Fine-Tuning aplikatu
5. ✅ Eredua ebaluatu
6. ✅ Emaitzak bistaratu eta interpretatu
7. ✅ Ondorioak eta hurrengo pausoak

## 🗂️ Dataset Aukeraketa

### 📦 Dataset Aukerak

1. **CIFAR-10**: 10 klaseko objektuak (60,000 irudi)
2. **Cats vs Dogs**: Katu eta txakurren klasifikazioa
3. **Fashion-MNIST**: Arropa klasifikazioa (10 klase)
4. **Flowers**: Lore mota desberdinak
5. **Custom Dataset**: Gure datuak

---

Proiektu honetarako **CIFAR-10** erabiliko dugu, baina edozein dataset erabil daiteke.

## 1️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegi nagusiak
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.applications import ResNet50, VGG16, MobileNetV2
from keras import layers, Model, callbacks
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Liburutegiak kargatuta!")
print(f"📦 TensorFlow bertsioa: {keras.__version__}")

## 2️⃣ Dataset-a Kargatu - CIFAR-10

In [ ]:
# CIFAR-10 dataset-a kargatu
print("📥 CIFAR-10 dataset-a kargatu...")
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Klaseen izenak
klase_izenak = ['Hegazkina', 'Autoa', 'Txoria', 'Katua', 'Orena', 
                'Txakurra', 'Igela', 'Zaldia', 'Ontzia', 'Kamoia']

print(f"\n📊 Dataset-aren informazioa:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  Klase kopurua: {len(klase_izenak)}")
print(f"  Irudi tamaina: {X_train.shape[1:3]}")

print("\n✅ Dataset-a kargatuta!")

## 3️⃣ Datu Esplorazioa

In [ ]:
# Klaseen banaketa
unique, counts = np.unique(y_train, return_counts=True)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Banaketa
axes[0].bar(range(len(klase_izenak)), counts, color='skyblue', edgecolor='black')
axes[0].set_xticks(range(len(klase_izenak)))
axes[0].set_xticklabels(klase_izenak, rotation=45, ha='right')
axes[0].set_ylabel('Irudi kopurua', fontsize=12)
axes[0].set_title('📊 Klaseen Banaketa (Train)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Irudi adibideak
axes[1].axis('off')
axes[1].set_title('🖼️ Dataset Adibideak', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Irudi adibideak klase bakoitzetik
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('🖼️ CIFAR-10 Irudi Adibideak', fontsize=16, fontweight='bold')

for i in range(10):
    # Klase bakoitzeko lehenengo irudia aurkitu
    idx = np.where(y_train == i)[0][0]
    
    ax = axes[i//5, i%5]
    ax.imshow(X_train[idx])
    ax.set_title(f'{klase_izenak[i]}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✅ Datu esplorazioa osatuta!")

## 4️⃣ Datuak Pre-prozesatu

In [ ]:
# Datuak normalizatu
X_train_norm = X_train.astype('float32') / 255.0
X_test_norm = X_test.astype('float32') / 255.0

# y kategoriko bihurtu
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Validation set sortu
from sklearn.model_selection import train_test_split
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_norm, y_train_cat, 
    test_size=0.2, 
    random_state=42
)

print("📊 Datuen banaketa:")
print(f"  Train: {X_train_final.shape}")
print(f"  Validation: {X_val.shape}")
print(f"  Test: {X_test_norm.shape}")

print("\n✅ Datuak pre-prozesatuta!")

## 5️⃣ Data Augmentation Konfiguratu

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

datagen.fit(X_train_final)

print("✅ Data Augmentation konfiguratuta!")

# Adibideak bistaratu
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('🔄 Data Augmentation Adibideak', fontsize=16, fontweight='bold')

# Irudi bat aukeratu
img = X_train_final[0:1]

for i, ax in enumerate(axes.flat):
    augmented = datagen.flow(img, batch_size=1)[0]
    ax.imshow(augmented[0])
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✅ Data Augmentation adibideak bistaratuta!")

## 6️⃣ Transfer Learning Eredua Sortu (ResNet50)

In [ ]:
# ResNet50 base model kargatu
print("🔄 ResNet50 base model-a kargatzen...")

# Irudiak 32x32 -> 96x96 resize egin behar dira (ResNet50-ek gutxienez 32x32 onartzen ditu)
# Baina CIFAR-10 32x32 da, beraz eredu txikiagoa erabiliko dugu

# VGG16 txikia edo custom CNN bat
def sortu_custom_cnn():
    model = keras.Sequential([
        # 1. Conv Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        
        # 2. Conv Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # 3. Conv Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Eredua sortu
model = sortu_custom_cnn()

# Konpilatu
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Laburpena
model.summary()

print("\n✅ CNN eredua sortuta eta konpilatuta!")
print(f"   Parametro kopurua: {model.count_params():,}")

## 7️⃣ Callbacks Konfiguratu

In [ ]:
# Callbacks zerrenda
callbacks_list = [
    # EarlyStopping
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # ModelCheckpoint
    callbacks.ModelCheckpoint(
        'best_cifar10_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # ReduceLROnPlateau
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Callbacks konfiguratuta!")

## 8️⃣ Eredua Entrenatu

In [ ]:
# Entrenamendua
print("🚀 Entrenamendua hasten...")

history = model.fit(
    datagen.flow(X_train_final, y_train_final, batch_size=64),
    epochs=50,
    steps_per_epoch=len(X_train_final) // 64,
    validation_data=(X_val, y_val),
    callbacks=callbacks_list,
    verbose=1
)

print("\n✅ Entrenamendua osatuta!")

## 9️⃣ Entrenamendua Bistaratu

In [ ]:
# Entrenamendua bistaratu
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_title('📊 Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('📉 Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Entrenamendua bistaratuta!")

## 🔟 Eredua Ebaluatu

In [ ]:
# Test datuak ebaluatu
print("📊 Test datuak ebaluatzen...")
test_loss, test_accuracy = model.evaluate(X_test_norm, y_test_cat, verbose=0)

print(f"\n📊 Test Emaitzak:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Aurreikuspenak
y_pred = model.predict(X_test_norm, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

print(f"\n✅ Test ebaluazioa osatuta!")

## 1️⃣1️⃣ Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=klase_izenak, yticklabels=klase_izenak)
plt.title('🎯 Confusion Matrix - CIFAR-10', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Aurreikusitako klasea', fontsize=12)
plt.ylabel('Benetako klasea', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("✅ Confusion matrix bistaratuta!")

## 1️⃣2️⃣ Classification Report

In [ ]:
# Classification report
print("📊 Classification Report:")
print("="*80)
print(classification_report(y_test, y_pred_classes, 
                          target_names=klase_izenak))
print("="*80)

## 1️⃣3️⃣ Aurreikuspenak Bistaratu

In [ ]:
# Aurreikuspen zuzenak eta okerrak
correct = np.where(y_pred_classes == y_test.flatten())[0]
incorrect = np.where(y_pred_classes != y_test.flatten())[0]

print(f"✅ Aurreikuspen zuzenak: {len(correct)} ({len(correct)/len(y_test)*100:.2f}%)")
print(f"❌ Aurreikuspen okerrak: {len(incorrect)} ({len(incorrect)/len(y_test)*100:.2f}%)")

# Aurreikuspen zuzenak
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('✅ Aurreikuspen Zuzenak', fontsize=16, fontweight='bold')

for i, idx in enumerate(correct[:10]):
    ax = axes[i//5, i%5]
    ax.imshow(X_test[idx])
    benetakoa = klase_izenak[y_test[idx][0]]
    aurreikuspena = klase_izenak[y_pred_classes[idx]]
    confidence = y_pred[idx][y_pred_classes[idx]]
    ax.set_title(f'{benetakoa}\n({confidence:.2f})', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

# Aurreikuspen okerrak
if len(incorrect) > 0:
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('❌ Aurreikuspen Okerrak', fontsize=16, fontweight='bold', color='red')
    
    for i, idx in enumerate(incorrect[:10]):
        ax = axes[i//5, i%5]
        ax.imshow(X_test[idx])
        benetakoa = klase_izenak[y_test[idx][0]]
        aurreikuspena = klase_izenak[y_pred_classes[idx]]
        confidence = y_pred[idx][y_pred_classes[idx]]
        ax.set_title(f'Benetakoa: {benetakoa}\nAurreikuspena: {aurreikuspena}\n({confidence:.2f})', 
                    fontsize=9, color='red')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

print("\n✅ Aurreikuspenak bistaratuta!")

## 1️⃣4️⃣ Eredua Gorde

In [ ]:
# Eredua gorde
model.save('cifar10_final_model.h5')
print("✅ Eredua gordeta: cifar10_final_model.h5")

# Ereduaren arkitektura JSON formatuan
import json
model_json = model.to_json()
with open('cifar10_model_architecture.json', 'w') as json_file:
    json_file.write(model_json)

print("✅ Ereduaren arkitektura gordeta: cifar10_model_architecture.json")

## 📝 Proiektuaren Ondorioak

### 🎯 Lorturiko Emaitzak

- **Dataset**: CIFAR-10 (10 klase, 60,000 irudi)
- **Eredua**: Custom CNN (3 Conv Block + Dense)
- **Test Accuracy**: ~75-80% (CIFAR-10 konplexua da)
- **Data Augmentation**: Overfitting-a ekiditeko
- **Callbacks**: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

### 📊 Ikasitako Gauzak

1. ✅ Dataset osoa kargatu eta pre-prozesatu
2. ✅ Data Augmentation aplikatu
3. ✅ Custom CNN eredua diseinatu
4. ✅ Callbacks erabiliz entrenamendua optimizatu
5. ✅ Eredua ebaluatu metrika desberdinekin
6. ✅ Emaitzak bistaratu eta interpretatu

### 🔑 Hobekuntzarako Iradokizunak

1. **Transfer Learning**: ResNet50, EfficientNet erabiliz
2. **Hiper-parametroen optimizazioa**: Grid Search, Bayesian Optimization
3. **Ensemble**: Eredu anitz konbinatu
4. **Dataset handiagoa**: ImageNet, COCO
5. **Objektu Detekzioa**: YOLO, Faster R-CNN

### 🚀 Hurrengo Pausoak

- **Proiektu errealak**: Custom dataset bat erabili
- **Deployment**: Flask, FastAPI web aplikazioa sortu
- **Mobile**: TensorFlow Lite mugikorreko aplikaziorako
- **Cloud**: AWS, GCP, Azure cloud deployment

---

## 🎓 Amaiera

**Zorionak!** Deep Computer Vision ikasketa-bidea osatu duzu! 🎉

Orain gai zara:
- CNN ereduak sortzeko eta entrenatzeko
- Transfer Learning erabiltzeko
- Objektu detekzioa aplikatzeko
- Aurpegi ezagutza sistemak garatzeko
- Proiektu osatuak garatzeko

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT

### 📧 Kontaktua

- **GitHub**: [github.com/maldalur](https://github.com/maldalur)
- **Email**: mikel.aldalur@example.com
- **LinkedIn**: [linkedin.com/in/mikelaldalur](https://linkedin.com)

---

**Eskerrik asko proiektu honetan parte hartzeagatik!** 🙏